In [1]:
import tabula
import numpy as np
import pandas as pd
import os 
from pathlib import Path
import PyPDF2
import re
import requests
import json
import time

In [23]:
# filenames = [
#              os.path.expanduser('/home/parth/Documents/USICT/it_res.pdf'),
#              os.path.expanduser('/home/parth/Documents/USICT/cse_res.pdf'),
#              os.path.expanduser('/home/parth/Documents/USICT/ece_res.pdf')]
filenames = [
    os.path.expanduser('~/Documents/USICT/ipu_results/cse_even_sems.pdf'),
    os.path.expanduser('~/Documents/USICT/ipu_results/ece_even_sems.pdf')
]

In [9]:
scheme_reg = re.compile(r'scheme\s+of\s+examinations',re.IGNORECASE)
institution_reg = re.compile(r'institution\s*:\s*([\w\n(,)& ]+)\nS\.No',re.IGNORECASE)
sem_reg = re.compile(r'se\s?m[.//\w\n]+:\s+([\w\n]+)',re.IGNORECASE)
programme_reg = re.compile(r'programme\s+name:\s+([\w(,)& \n]+)SchemeID',re.IGNORECASE)
branch_reg = re.compile(r'[\w &]+\(([\w ]+)\)')

In [10]:
def get_info(text) :
    college = institution_reg.search(text)[1].replace('\n','').strip().title()
    semester = int(sem_reg.search(text)[1].replace('\n','').strip())
    course = programme_reg.search(text)[1].replace('\n','').strip().title()
    branch = branch_reg.search(course)[1].strip().title()
    course = course[0:course.find('(')].strip()
    info = {
        'college' : college,
        'semester' : semester,
        'course' : course,
        'branch' : branch,
    }
    return info

In [29]:
# SITE = "https://cgpa-book.herokuapp.com/academia"
SITE = "http://localhost:3000/academia"

In [31]:
#Add college
data ={ 
    'college' : {
        'college' : "University School Of Information, Communication & Technology (Formerly Usit)"
        }
}
r = requests.post(SITE+"/college",json=data)
print(r,r.content)


<Response [503]> b'{"message":"sw is not defined"}'


In [11]:
def already_exists(info) :
    r = requests.get(SITE+"/semester",params=info)
    content = json.loads(r.content)
#     print(r.status_code,r.content)
    return r.status_code == 200 and content != {}
    

In [12]:
def getSubjects(df) :
    subjects = []
    for index,row in df.iterrows() :
        subject = {}
        subject['subject'] = row['Subject'].strip().title()
        subject['subjectCode'] = row['Code']
        subject['credits'] = row['Credit']
        subjects.append(subject)
    return subjects

In [27]:
for filename in filenames :
    pdf = PyPDF2.PdfFileReader(filename)
    print(filename,pdf.getNumPages())
    
    for i in range(0,pdf.getNumPages()) :
        text = pdf.getPage(i).extractText()
        if scheme_reg.search(text) :
            info = get_info(text)
            df = tabula.read_pdf(filename,pages=i+1)
            subjects = getSubjects(df[0])        
            if already_exists(info) :
                print("information already exists")
                continue
            info['semester'] = {'semester' : info['semester'], 'subjects' : subjects}
            r = requests.post(SITE+"/semester",json=info)
            print(r,r.content)
            time.sleep(2)
#             print(info)

    

/home/parth/Documents/USICT/ipu_results/cse_even_sems.pdf 31
<Response [400]> b'{"type":"ValueError","message":"college not found"}'
<Response [400]> b'{"type":"ValueError","message":"college not found"}'


KeyboardInterrupt: 

In [ ]:
from IPython.display import display